In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [18,10]

### Dominant colors in images
- All images consist of pixels 
- Each pixel has three values :RGB
- Perform k-means on standardized RGB values to find cluster centers
- Uses: Identify features in satellite images

### Tools to find dominant colors
- Convert image to pixels: `matplotlib.image.imread`
- Display colors of cluster centers: `matplotlib.pyplot.imshow`

In [3]:
import matplotlib.image as img
image = img.imread('data/sea.jpg')
image.shape

(810, 1305, 3)

In [4]:
r = []
g = []
b = []

for row in image:
    for pixel in row: 
        # A pixel contains RGB values
        temp_r, temp_g, temp_b = pixel
        r.append(temp_r)
        g.append(temp_g)
        b.append(temp_b)

In [5]:
from scipy.cluster.vq import whiten

In [6]:
pixels = pd.DataFrame({'red': r,
                      'blue': b,
                      'green': g,
                      'scaled_red': whiten(r),
                      'scaled_green': whiten(g),
                      'scaled_blue': whiten(b)})

In [7]:
from scipy.cluster.vq import kmeans, vq

In [ ]:
distortions = []
num_clusters = range(1, 11)

# Create a list of distortions from the kmeans method
for i in num_clusters: 
    cluster_centers, distortion = kmeans(pixels[['scaled_red', 'scaled_blue', 'scaled_green']], i)
    distortions.append(distortion)
    
# Create a data frame with two list - number of clusters and distortions 
elbow_plot = pd.DataFrame({'num_clusters': num_clusters,
                          'distortions': distortions})

# Create a line plot of num_clusters and distortions
sns.lineplot(x='num_clusters', y='distortions', data = elbow_plot)
plt.xticks(num_clusters)
plt.show()

### Find dominant colors

In [ ]:
cluster_centers, _ = kmeans(pixels[['scaled_red', 'scaled_blue', 'scaled_green']], 2)

In [ ]:
colors = []

# Find Standard Deviations 
r_std, g_std, b_std = pixels[['red', 'green', 'blue']].std()

# Scale actual RGB values in range of 0-1
for cluster_center in cluster_centers:
    scaled_r, scaled_g, scaled_b = cluster_center
    colors.append((
        scaled_r * r_std/255,
        scaled_g * g_std/255,
        scaled_b * b_std/255
    ))

### Display dominant colors

In [ ]:
#Dimensions: 2 x 3 (N x 3 matrix)
print(colors)

In [ ]:
#Dimensions 1 x 2 x 3 (1 x N x 3 matrix)
plt.imshow([colors])
plt.show()

In [ ]:
r = []
g = []
b = []

In [ ]:
# Import image class of matplotlib
import matplotlib.image as img

# Read batman image and print dimensions
batman_image = img.imread('data/batman.jpg')
print(batman_image.shape)

# Store RGB values of all pixels in lists r, g and b
for pixel in batman_image:
    for temp_r, temp_g, temp_b in pixel:
        r.append(temp_r)
        g.append(temp_g)
        b.append(temp_b)

In [ ]:
batman_df = pd.DataFrame({'red': r,
                         'green': g,
                         'blue': b,
                         'scaled_red': whiten(r),
                         'scaled_green': whiten(g),
                         'scaled_blue': whiten(b)})

In [ ]:
distortions = []
num_clusters = range(1, 7)

# Create a list of distortions from the kmeans function
for i in num_clusters:
    cluster_centers, distortion = kmeans(batman_df[['scaled_red', 'scaled_blue','scaled_green']], i)
    distortions.append(distortion)

# Create a data frame with two lists, num_clusters and distortions
elbow_plot = pd.DataFrame({'num_clusters': num_clusters,
                         'distortions': distortions})

# Create a line plot of num_clusters and distortions
sns.lineplot(x='num_clusters', y='distortions', data = elbow_plot)
plt.xticks(num_clusters)
plt.show()

In [ ]:
cluster_centers, _ = kmeans(batman_df[['scaled_red', 'scaled_green', 'scaled_blue']], 3)
colors = []

In [ ]:
# Get standard deviations of each color
r_std, g_std, b_std = batman_df[['red', 'green', 'blue']].std()

for cluster_center in cluster_centers:
    scaled_r, scaled_g, scaled_b = cluster_center
    # Convert each standardized value to scaled value
    colors.append((
        scaled_r * r_std / 255,
        scaled_g * g_std / 255,
        scaled_b * b_std / 255
    ))

# Display colors of cluster centers
plt.imshow([colors])
plt.show()

### Document clustering
1. Clean data before processing
2. Determine the importance of the terms in a document (in TF-IDF matrix)
3. Cluster the TF-IDF matrix
4. Find top terms, documents in each cluster

### Clean and tokenize data
- Convert text to into smaller parts called tokens, clean data for processing

In [ ]:
from nltk.tokenize import word_tokenize
import re

def remove_noise(text, stop_words = []):
    tokens = word_tokenize(text)
    cleaned_tokens = []
    for token in tokens:
        token = re.sub('[^A-Za-z0-9]+', '', token)
        if len(token) > 1 and token.lower() not in stop_words:
            #Get lowercase
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

data = remove_noise("It is lovely weather we are having.\
              I hope the weather continues.",
             stop_words = ['it', 'is', 'we', 'the'])
data

### Document term matrix and sparse matrices
- Document term matrix formed
- Most elements in matrix are zero

![matrix](data/matrix.png)

- Sparse matrix is created

![sparse_matrix](data/sparse_matrix.png)

### TF-IDF(Term Frequency - Inverse Document Frequency)
- A weighted measure: evaluate how important a word is to a document in a collection

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=50,
                                 min_df=0.2, tokenizer=remove_noise)

tfidf_matrix = tfidf_vectorizer.fit_transform(data)

### Clustering with sparse matrix
- `kmeans` in SciPy does not support sparse matrices
- Use `.todense()` to convert a matrix

In [ ]:
cluster_centers, distortion = kmeans(tfidf_matrix.todense(), 2)

### Top terms per cluster
- Cluster centers: lists with a size 